In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import math
from functools import partial
from open3d.t.geometry import TriangleMesh
from tqdm import tqdm 
from scipy.interpolate import UnivariateSpline
from scipy.optimize import least_squares
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
dataname = "/home/chris/Code/PointClouds/data/FLIPscans/MortenPlateTopSuperCleaned500k.ply"
pcd = o3d.io.read_point_cloud(dataname)

In [3]:
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

PointCloud with 493763 points.

In [4]:
#Outlier removal

nn = 16
std_multiplier = 10

filtered_pcd = pcd.remove_statistical_outlier(nn,std_multiplier)
outliers = pcd.select_by_index(filtered_pcd[1], invert = True)
outliers.paint_uniform_color([1,0,0])
filtered_pcd = filtered_pcd[0]

#o3d.visualization.draw_geometries([filtered_pcd, outliers])

In [5]:
#Downsampling

voxel_size = 0.01
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size=voxel_size)
print(f'number of points: {len(pcd_downsampled.points)}')
o3d.visualization.draw_geometries([pcd_downsampled])

number of points: 493486


In [6]:
#Extract normals

nn_distance = np.mean([pcd.compute_nearest_neighbor_distance()])
radius_normals = nn_distance*4

pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)

pcd_downsampled.paint_uniform_color([0.6,0.6,0.6])
#o3d.visualization.draw_geometries([pcd_downsampled, outliers])

PointCloud with 493486 points.

In [7]:
front =  [ 0.0, 0.0, 1.0 ]
lookat = [ 4.155493041143778, 3.3619307090130235, 0.041189902146896884 ]
up =  [ 0.0, 1.0, 0.0 ]
zoom = 0.61999999999999988

pcd = pcd_downsampled

#o3d.visualization.draw_geometries([pcd_downsampled], zoom=zoom, front=front, lookat=lookat, up=up)

In [8]:
# Multi-order Ransac

max_plane_idx = 1
pt_to_plane_dist = 0.4

segment_models = {}
segments = {}
main_surface_idx = 0
largest_surface_points = 0
rest = pcd

for i in range(max_plane_idx):
    #print(f'Run {i}/{max_plane_idx} started. ', end='')
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=50000)
    segments[i] = rest.select_by_index(inliers)
    if len(segments[i].points) > largest_surface_points:
        largest_surface_points = len(segments[i].points) 
        main_surface_idx = i
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print('Done')

#print('Largest surface found with segment idx', main_surface_idx)
print(segment_models)
o3d.visualization.draw_geometries([segments[main_surface_idx]],zoom=zoom,front=front,lookat=lookat,up=up)

Done
{0: array([-6.82620095e-04,  8.16988533e-03,  9.99966393e-01,  1.03032875e-04])}


In [9]:
# Gap detection
main_surface_pcd = segments[main_surface_idx]
np.asarray(main_surface_pcd.points)[:, 2] = 0

points = np.asarray(main_surface_pcd.points)
pcd_tree = o3d.geometry.KDTreeFlann(main_surface_pcd)

grid_resolution = 0.1
search_radius = 0.5
neighbor_threshold = 4

min_bound = main_surface_pcd.get_min_bound()
max_bound = main_surface_pcd.get_max_bound()

x_grid = np.arange(min_bound[0], max_bound[0], grid_resolution)
y_grid = np.arange(min_bound[1], max_bound[1], grid_resolution)
z_value = np.mean([min_bound[2], max_bound[2]])

# Generate all query points at once
xv, yv = np.meshgrid(x_grid, y_grid, indexing='ij')
query_points = np.vstack((xv.ravel(), yv.ravel(), np.full_like(xv.ravel(), z_value))).T

gap_points = []

# Efficient neighbor search
for query_point in tqdm(query_points):
    k, idx, _ = pcd_tree.search_radius_vector_3d(query_point, search_radius)
    if k < neighbor_threshold:
        gap_points.append(query_point)

gap_pcd = o3d.geometry.PointCloud()
gap_pcd.points = o3d.utility.Vector3dVector(gap_points)
gap_pcd.paint_uniform_color([1, 0, 0])

o3d.visualization.draw_geometries([main_surface_pcd, gap_pcd], zoom=zoom, front=front, lookat=lookat, up=up)

100%|██████████| 6193050/6193050 [00:39<00:00, 156530.57it/s]


In [10]:
#DBScan clustering

eps = 0.30
min_points = 10

labels = np.array(gap_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
print(labels[0])
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[labels]
colors[labels<0] = 0
gap_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
# o3d.visualization.draw_geometries([gap_pcd], zoom=zoom, front=front, lookat=lookat, up=up)

Precompute neighbors.[========================================] 100%
point cloud has 103 clusters======================>] 97%
0


In [11]:
#Filter small clusters

points = np.asarray(gap_pcd.points)
colors = np.asarray(gap_pcd.colors)  
point_threshold = 50
mask = np.zeros(len(points), dtype=bool)
unique_labels = np.unique(labels)

filtered_pcd = o3d.geometry.PointCloud()
filtered_clusters = []
filtered_labels_unique = []
filtered_labels_vector = []

#Filter and assign
for label in unique_labels:
    if label != -1:
        cluster_indices = np.where(labels == label)[0]
        if len(cluster_indices) >= point_threshold:
            mask[cluster_indices] = True
            cluster_points = points[cluster_indices]
            filtered_labels_unique.append(label)

for label in filtered_labels_unique:
    cluster_indices = np.where(labels == label)[0]
    cluster_points = points[cluster_indices]
    filtered_clusters.append(cluster_points)
    cluster_labels = np.tile(label, (cluster_points.shape[0], 1))
    filtered_labels_vector.append(cluster_labels)

total_points = np.vstack([cluster for cluster in filtered_clusters])
filtered_labels = np.vstack([cluster_labels for cluster_labels in filtered_labels_vector]).squeeze()

#print(total_points.shape)
#print(filtered_labels.shape)

filtered_pcd.points = o3d.utility.Vector3dVector(total_points)

#Recolor the filtered pcd
max_label = filtered_labels.max()
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[filtered_labels]
filtered_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([filtered_pcd], zoom=zoom, front=front, lookat=lookat, up=up)

In [ ]:
def reset_camera(view_ctl):
    # Set camera view parameters (adjust these as needed)
    view_ctl.set_lookat([0, 0, 0])  # Center of the view
    view_ctl.set_up([0, 1, 0])  # Up direction
    view_ctl.set_front([0, 0, -1])  # Front direction
    view_ctl.set_zoom(1.0)  # Zoom level

visualization = o3d.visualization.VisualizerWithKeyCallback()
visualization.create_window()
visualization.get_render_option().background_color = np.asarray([0.95, 0.95, 0.95])

view_ctl = visualization.get_view_control()
#reset_camera(view_ctl)

class CallbackState:
    def __init__(self, initial_labels, pcd, error_color_list, density_color_list):
        self.initial_pcd = pcd
        self.initial_labels = initial_labels
        self.working_pcd = o3d.geometry.PointCloud()
        self.initial_points = np.asarray(pcd.points)
        self.unique_labels = np.unique(self.initial_labels)
        self.error_color_list = error_color_list
        self.density_color_list = density_color_list
        self.geometry_added = False
        self.key_pressed = False
        self.iterations = 10
        self.tolerance = 0.001
        self.total_error_dict = {}
        self.density_dict = {}
        self.cluster_color_list = []
        self.circle_geometries = []
        self.circles_on = False
        self.error_visibility = False
        self.density_visibility = False
        self.processing_done = False

    def single_distance(self, point, center):
        return np.sqrt((point[0] - center[0])**2 + (point[1] - center[1])**2)
    
    def distances_from_center(self, cluster_points, center):
        return np.sqrt((cluster_points[:, 0] - center[0])**2 + (cluster_points[:, 1] - center[1])**2)
    
    def levenberg_marquardt(self, cluster_points, p_vec, iterations, tolerance):
        damp_lambda = 0.01
        p_vec = np.array(p_vec)

        for _ in range(iterations):
            x_c, y_c, r = p_vec
            xs, ys = cluster_points[:, 0], cluster_points[:, 1]
            distances = np.sqrt((xs - x_c)**2 + (ys - y_c)**2)
            residuals = distances - r

            #safe_distances = np.where(distances == 0, np.finfo(float).eps, distances)
            J_x = -2 * (distances - r) * (xs - x_c) / distances
            J_y = -2 * (distances - r) * (ys - y_c) / distances
            J_r = 2 * (r - distances)

            Jacobian = np.vstack([J_x, J_y, J_r]).T

            hessian = Jacobian.T @ Jacobian
            gradient = Jacobian.T @ residuals
            p_change = np.linalg.inv(hessian + damp_lambda * np.eye(3)) @ gradient
            updated_p_vec = p_vec - p_change
            new_residuals = self.distances_from_center(cluster_points, updated_p_vec[:2]) - updated_p_vec[2]
            new_total_error = np.sum(new_residuals**2)

            if new_total_error < np.sum(residuals**2):
                damp_lambda /= 10
                p_vec = updated_p_vec
                if np.abs(np.sum(residuals**2) - new_total_error) < tolerance:
                    break
            else:
                damp_lambda *= 10

        inlier_mask = distances <= 1.75 * p_vec[2]
        inliers = cluster_points[inlier_mask]
        if len(inliers) > 0:
            new_radius = np.max(np.sqrt((inliers[:, 0] - p_vec[0])**2 + (inliers[:, 1] - p_vec[1])**2))
            p_vec[2] = new_radius

        return p_vec, new_total_error

    def paint_dark(self, vis):
        grey_colors = np.tile((0.1,0.1,0.1), (self.initial_points.shape[0], 1))
        self.working_pcd.colors = o3d.utility.Vector3dVector(grey_colors)
        vis.update_geometry(self.working_pcd)

    def toggle_error_visibility(self, vis):
        if not self.processing_done:
            return
        else:
            if self.error_visibility:
                self.paint_dark(vis)
                self.error_visibility = False
            else:
                if hasattr(self, 'total_error_colors'):
                    self.working_pcd.colors = o3d.utility.Vector3dVector(self.total_error_colors)
                    vis.update_geometry(self.working_pcd)   
                else:
                    self.cluster_color_list.clear()
                    error_values = [error for _, error in self.total_error_dict.items()]
                    min_error, max_error = min(error_values), max(error_values)
                    step = (max_error - min_error) / n_bins if error_values else 0
                    #print(f'Min error: {min_error}, max error: {max_error}, step: {step}')

                    for label in self.unique_labels:
                        if label in self.total_error_dict:
                                cluster_indices = np.where(self.initial_labels == label)[0]
                                error = self.total_error_dict[label]
                                percentile = int((error - min_error) // step)
                                percentile = min(percentile, n_bins - 1)
                                cluster_color = self.error_color_list[percentile]
                                cluster_color_points = np.tile(cluster_color, (cluster_indices.shape[0] ,1))
                                self.cluster_color_list.append(cluster_color_points)

                    total_colors = np.vstack([cluster for cluster in self.cluster_color_list])
                    self.total_error_colors = total_colors
                    self.working_pcd.colors = o3d.utility.Vector3dVector(total_colors)
                    vis.update_geometry(self.working_pcd)
                self.error_visibility = True
                self.density_visibility = False
    
    def toggle_density_visibility(self, vis):
        if not self.processing_done:
            return
        else:
            if self.density_visibility:
                self.paint_dark(vis)
                self.density_visibility = False
            else:
                if hasattr(self, 'total_density_colors'):
                    self.working_pcd.colors = o3d.utility.Vector3dVector(self.total_density_colors)
                    vis.update_geometry(self.working_pcd)   
                else:
                    self.cluster_color_list.clear()
                    densities = list(self.density_dict.values())
                    min_density, max_density = min(densities), max(densities)
                    step = (max_density - min_density) / n_bins if densities else 0

                    for label in self.unique_labels:
                        if label in self.density_dict:
                            cluster_indices = np.where(self.initial_labels == label)[0]
                            density = self.density_dict[label]
                            percentile = int((density - min_density) // step)
                            percentile = min(percentile, n_bins - 1)
                            cluster_color = self.density_color_list[percentile]
                            cluster_color_points = np.tile(cluster_color, (cluster_indices.shape[0], 1))
                            self.cluster_color_list.append(cluster_color_points)
                        
                    total_colors = np.vstack([cluster for cluster in self.cluster_color_list])
                    self.total_density_colors = total_colors
                    self.working_pcd.colors = o3d.utility.Vector3dVector(total_colors)
                    vis.update_geometry(self.working_pcd)
                self.density_visibility = True
                self.error_visibility = False

            
    def calculate_density(self, cluster_points, center, radius):
        distances = self.distances_from_center(cluster_points, center)
        insiders = np.sum(distances <= radius)
        circle_area = np.pi*(radius **2)
        density = insiders / circle_area if circle_area > 0 else 0 
        return density

    def create_circle(self, center, radius, num_segments=100):
        angles = np.linspace(0, 2 * np.pi, num_segments, endpoint=False)
        x = center[0] + radius * np.cos(angles)
        y = center[1] + radius * np.sin(angles)
        z = np.zeros_like(x)  # Assuming the circle is in the XY plane
        points = np.vstack((x, y, z)).T
        lines = [[i, (i + 1) % num_segments] for i in range(num_segments)]
        colors = [[0.1, 0.1, 0.9] for _ in range(num_segments)]  # Green color for the circle
        return points, lines, colors
    
    def circle_for_visualizer(self, center, radius):
        points, lines, colors = self.create_circle(center, radius)
        circle = o3d.geometry.LineSet()
        circle.points = o3d.utility.Vector3dVector(points)
        circle.lines = o3d.utility.Vector2iVector(lines)
        circle.colors = o3d.utility.Vector3dVector(colors)
        return circle
    
    def toggle_circles_visibility(self, vis):
        if not self.processing_done:
            return
        else:
            if len(self.circle_geometries) != 0:
                if self.circles_on:
                    for circle in self.circle_geometries:
                        vis.remove_geometry(circle, reset_bounding_box = False)
                    self.circles_on = False
                else: 
                    for circle in self.circle_geometries:
                        vis.add_geometry(circle, reset_bounding_box = False)
                    self.circles_on = True

    def fit_circles(self, vis):
        if self.key_pressed:
            return
        self.key_pressed = True

        for label in self.unique_labels:  # Assumes labels are 0-indexed
            cluster_indices = np.where(self.initial_labels == label)[0]
            cluster_points = self.initial_points[cluster_indices]

            centroid = np.mean(cluster_points, axis=0)
            distances = self.distances_from_center(cluster_points, centroid)
            initial_radius = np.mean(distances)
            p_vec = [centroid[0], centroid[1], initial_radius]
            estimates, error = self.levenberg_marquardt(cluster_points, p_vec, self.iterations, self.tolerance)
            self.total_error_dict[label] = error
            center_estimate = estimates[:2]
            radius_estimate = estimates[2]
            
            circle_geom = self.circle_for_visualizer(center_estimate, radius_estimate)
            self.circle_geometries.append(circle_geom)

            cluster_density = self.calculate_density(cluster_points, center_estimate, radius_estimate)
            self.density_dict[label] = cluster_density
            #cluster_color = self.error_color_list[label % len(self.error_color_list)]
            #cluster_color_points = np.tile(cluster_color, (cluster_points.shape[0], 1))

            if not self.geometry_added:
                self.working_pcd.points = o3d.utility.Vector3dVector(cluster_points)
                #self.working_pcd.colors = o3d.utility.Vector3dVector(cluster_color_points)
                vis.add_geometry(self.working_pcd)
                self.geometry_added = True
            else:
                current_points = np.asarray(self.working_pcd.points)
                current_colors = np.asarray(self.working_pcd.colors)

                total_points = np.vstack((current_points, cluster_points))
                #total_colors = np.vstack((current_colors, cluster_color_points))

                self.working_pcd.points = o3d.utility.Vector3dVector(total_points)
                #self.working_pcd.colors = o3d.utility.Vector3dVector(total_colors)

            vis.update_geometry(self.working_pcd)
            vis.poll_events()
            vis.update_renderer()
            time.sleep(0.03)  # Shorter sleep to reduce blocking time

        vis.poll_events()
        vis.update_renderer()
        self.processing_done = True

colors = ["green", "yellow", "red"]
n_bins = 10
cmap = LinearSegmentedColormap.from_list("custom_green_yellow_red", colors, N=n_bins)
error_color_list = [cmap(i/n_bins)[:3] for i in range(n_bins)]

densities_colormap = plt.get_cmap('viridis', n_bins)
density_color_list = [densities_colormap(i/n_bins)[:3] for i in range(n_bins)]

state = CallbackState(initial_labels=filtered_labels, pcd=filtered_pcd, error_color_list=error_color_list, density_color_list=density_color_list)
visualization.register_key_callback(262, state.fit_circles)
visualization.register_key_callback(265, state.toggle_circles_visibility)
visualization.register_key_callback(263, state.toggle_error_visibility)
visualization.register_key_callback(264, state.toggle_density_visibility)

visualization.run()
visualization.destroy_window()